In [2]:
from sklearn.datasets import (
    load_iris, load_wine,
    load_digits, load_breast_cancer
)
import pandas as pd
import os, sys
import time
import gc
import tracemalloc
# asumiendo que iris.ipynb está en notebooks/, y somJ/ en la carpeta superior:
sys.path.insert(0, os.path.abspath(".."))

from somJ.k_fold import cross_val_som_metrics
import somJ.config as config
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, load_wine, load_digits, load_breast_cancer
from sklearn.datasets import fetch_openml, fetch_covtype, fetch_20newsgroups_vectorized
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.datasets import load_digits, load_iris, fetch_20newsgroups_vectorized, make_swiss_roll, make_s_curve
from tensorflow.keras.datasets import fashion_mnist, mnist, cifar10
import numpy as np
import pandas as pd
from scipy.io import loadmat

2025-05-07 16:31:55.858467: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-07 16:31:55.865411: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-07 16:31:55.918575: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-07 16:31:55.965019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746631916.007825    9573 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746631916.02

In [3]:
def reduce_mem_usage(df):
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min >= -128 and c_max <= 127:
                    df[col] = df[col].astype('int8')
                elif c_min >= -32768 and c_max <= 32767:
                    df[col] = df[col].astype('int16')
                elif c_min >= -2147483648 and c_max <= 2147483647:
                    df[col] = df[col].astype('int32')
                else:
                    df[col] = df[col].astype('int64')
            else:
                df[col] = df[col].astype('float32') 
    return df


In [4]:

def load_dataset(name):
    if name == 'Iris':
        data = load_iris()
        X, y = data.data, data.target

    elif name == 'Digits':
        data = load_digits()
        X, y = data.data, data.target

    elif name == 'MNIST':
        (X_train, y_train), _ = mnist.load_data()
        X = X_train.reshape((X_train.shape[0], -1))
        y = y_train

    elif name == 'Fashion MNIST':
        (X_train, y_train), _ = fashion_mnist.load_data()
        X = X_train.reshape((X_train.shape[0], -1))
        y = y_train

    elif name == 'CIFAR-10':
        (X_train, y_train), _ = cifar10.load_data()
        X = X_train.reshape((X_train.shape[0], -1))
        y = y_train.flatten()

    elif name == 'Swiss Roll':
        X, y = make_swiss_roll(n_samples=2000)
        
    elif name == 'S-curve':
        X, y = make_s_curve(n_samples=2000)
        
    elif name == '20 Newsgroups':
        news = fetch_20newsgroups_vectorized()
        X, y = news.data.toarray(), news.target

    elif name == 'COIL-20':
        # Suponiendo que tienes el archivo coil-20.mat descargado en tu directorio
        mat = loadmat('coil-20.mat')
        X = mat['X']  # Ajustar si la clave es diferente
        y = mat['y'].flatten()

    else:
        raise ValueError(f"Dataset {name} no reconocido.")

    # Convertir a DataFrame para reducir memoria si es necesario
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)
    X = reduce_mem_usage(X)
    return X.values, y

In [5]:
def evaluate(name,X,y,memory=False):
    if memory: tracemalloc.start()  # Inicia medición de memoria
    start = time.perf_counter()
    metrics = cross_val_som_metrics(
        X, y,
        k=5,
        method="random",
        update="batchmap",
        config=config,
        random_state=42,
        scale=True,
        prog_bar=True
    )
    gc.collect()
    end = time.perf_counter()
    peak=0
    if memory:
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
    print(
        f"{name:15s} | "
        f"Acc: {metrics['accuracy']:.2f}  | "
        f"Prec: {metrics['precision']:.2f}  | "
        f"Rec: {metrics['recall']:.2f}  | "
        f"F1: {metrics['f1']:.2f}  | "
        f"Time: {end - start:.2f}s  | "
        f"Memory: {peak / 1024 / 1024:.2f} MB  | "  # En MB
        f"Trustworthiness: {metrics['tw']:.2f}  | "
        f"Correlation: {metrics['corr']:.2f}"
    )

Iris            | Acc: 0.99  | Prec: 0.99  | Rec: 0.99  | F1: 0.99  | Time: 0.15s  | Memory: 0.00 MB  | Trustworthiness: 0.00  | Correlation: 0.00
Digits          | Acc: 0.75  | Prec: 0.76  | Rec: 0.75  | F1: 0.74  | Time: 0.56s  | Memory: 0.00 MB  | Trustworthiness: 0.00  | Correlation: 0.00
MNIST           | Acc: 0.51  | Prec: 0.51  | Rec: 0.50  | F1: 0.48  | Time: 69.54s  | Memory: 0.00 MB  | Trustworthiness: 0.00  | Correlation: 0.00
Fashion MNIST   | Acc: 0.58  | Prec: 0.59  | Rec: 0.58  | F1: 0.56  | Time: 73.47s  | Memory: 0.00 MB  | Trustworthiness: 0.00  | Correlation: 0.00

In [6]:
dataset_names = [
    "Iris",
    "Digits",
    "MNIST",
    "Fashion MNIST",
    # "20 Newsgroups",
    # "COIL-20",
    # "CIFAR-10"
]

# Loop general: procesar y liberar cada dataset
for name in dataset_names:
    X, y = load_dataset(name)
    # Escalado
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    evaluate(name,X_scaled,y,False)
    del X, y, X_scaled, scaler
    gc.collect()

Entrenando SOM:   0%|                              | 0/50 [00:00<?, ?it/s, lr=0.00863, radius=0.294]


Iris            | Acc: 1.00  | Prec: 1.00  | Rec: 1.00  | F1: 1.00  | Time: 1.24s  | Memory: 0.00 MB  | Trustworthiness: 0.00  | Correlation: 0.00


Entrenando SOM:   0%|                              | 0/50 [00:02<?, ?it/s, lr=0.00863, radius=0.294]


Digits          | Acc: 0.97  | Prec: 0.97  | Rec: 0.97  | F1: 0.97  | Time: 14.47s  | Memory: 0.00 MB  | Trustworthiness: 0.00  | Correlation: 0.00


Entrenando SOM:   0%|                              | 0/50 [05:39<?, ?it/s, lr=0.00863, radius=0.294]


MNIST           | Acc: 0.86  | Prec: 0.86  | Rec: 0.86  | F1: 0.86  | Time: 1700.94s  | Memory: 0.00 MB  | Trustworthiness: 0.00  | Correlation: 0.00


Entrenando SOM:   0%|                              | 0/50 [06:03<?, ?it/s, lr=0.00863, radius=0.294]


Fashion MNIST   | Acc: 0.74  | Prec: 0.74  | Rec: 0.74  | F1: 0.74  | Time: 1811.95s  | Memory: 0.00 MB  | Trustworthiness: 0.00  | Correlation: 0.00
